# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115736e+02     1.389534e+00
 * time: 0.09170293807983398
     1     1.024760e+01     8.309271e-01
 * time: 1.3411879539489746
     2    -1.159678e+01     1.006406e+00
 * time: 1.5046229362487793
     3    -3.412130e+01     7.535815e-01
 * time: 1.6485259532928467
     4    -4.737159e+01     5.703962e-01
 * time: 1.8108620643615723
     5    -5.687580e+01     2.339000e-01
 * time: 1.9608519077301025
     6    -5.968744e+01     1.759781e-01
 * time: 2.0677499771118164
     7    -6.083949e+01     7.750590e-02
 * time: 2.185215950012207
     8    -6.134030e+01     4.156983e-02
 * time: 2.29276704788208
     9    -6.164230e+01     3.868859e-02
 * time: 2.42930006980896
    10    -6.182341e+01     3.144549e-02
 * time: 2.5384199619293213
    11    -6.194276e+01     2.742578e-02
 * time: 2.6478350162506104
    12    -6.203312e+01     2.115860e-02
 * time: 2.7664270401000977
    13    -6.210295e+01     1.601350e-02
 * time: 2.87508296966552

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671076
    AtomicLocal         -18.8557676
    AtomicNonlocal      14.8522631
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336824

    total               -62.261666461478
